In [1]:
import numba
from numba import cuda, float64
import numpy as np
import math
from scipy.integrate import quad
from tqdm import tqdm
from celerite import terms
import celerite
import time as tempo

In [2]:
# Function to calculate the radius of the innermost stable circular orbit (ISCO)
def R_ISCO(M, chi):
    Z1 = 1. + (1. - chi**2)**(1/3) * ((1. + chi)**(1/3) + (1. - chi)**(1/3))
    Z2 = np.sqrt(3. * chi**2 + Z1**2)
    isco_ = 3 + Z2 - np.sign(chi) * np.sqrt((3. - Z1) * (3. + Z1 + 2. * Z2))
    return isco_ * G * M / c**2

In [3]:
# Function to de-evolve the phase of an orbiting object
# Find where the emitting element of the accretion disc was at an arbitrary time
# Modifies a pre-allocated CUDA device array (filled with zeros) with the de-evolved angles
@cuda.jit(device = True)
def de_evolve(xi, theta, time, initial_phase, M):
    td = time_delay(xi, theta, inc)
    kf = keplerian_freq(M, xi) 
    return (theta - (time -td)*kf + initial_phase) - (2.*math.pi)*math.floor((theta - (time -td)*kf + initial_phase)/(2.*math.pi))

#Function to compute time delay
# Modifies a pre-allocated CUDA device array (filled with zeros) with the de-evolved angles
# @cuda.jit(device=True) would be more efficient if we don't need to save them in a device array
@cuda.jit(device=True)
def time_delay(xi, theta, inc):
    return xi*(1+math.sin(inc)*math.cos(theta))

#x, y are separated cuda device
@cuda.jit(device = True)
def position_components(angle, R):
    x = R * math.sin(angle)  # x-coordinate
    y = R * math.cos(angle)  # y-coordinate
    return (x,y)

#vx, vy are separated cuda device
@cuda.jit(device = True)
def velocity_components(angle, R, M, corot=0):
    v_kep = math.sqrt(G * M / R)  # Keplerian velocity
    vx = v_kep * math.cos(angle)  # x-component of velocity
    vy = -v_kep * math.sin(angle) # y-component of velocity

    if corot == 0:  # Corotating case
        return (vx, vy)
    else:  # Non-corotating case
        return (-vx, -vy)

@cuda.jit(device = True)
def keplerian_freq(M, R):
    omega_k = math.sqrt(G * M/ R**3)
    return omega_k

@cuda.jit(device=True)
def doppler_factor(v1, v2, r1, r2):
    #norms/dots v1, v2
    norm_v1_sq = v1[0]*v1[0] + v1[1]*v1[1]#dot_product(v1, v1)#v1[0]*v1[0] + v1[1]*v1[1]
    norm_v2_sq = v2[0]*v2[0] + v2[1]*v2[1]#dot_product(v2, v2) #v2[0]*v2[0] + v2[1]*v2[1]
    v2_dot_v1 = v2[0]*v1[0] + v2[1]*v1[1]#dot_product(v2, v1) #v2[0]*v1[0] + v2[1]*v1[1]
    norm_v1_minus_v2_sq = (v1[0]-v2[0])*(v1[0]-v2[0]) + (v1[1]-v2[1])*(v1[1]-v2[1])#dot_product(v1-v2, v1-v2) #(v1[0]-v2[0])*(v1[0]-v2[0]) + (v1[1]-v2[1])*(v1[1]-v2[1])
    
    # Parallel components of v2
    v2_para_x = (v2_dot_v1 / norm_v1_sq) * v1[0]
    v2_para_y = (v2_dot_v1 / norm_v1_sq) * v1[1]

    # Perpendicular components of v2
    v2_perp_x = v2[0] - v2_para_x
    v2_perp_y = v2[1] - v2_para_y

    # Lorentz factor v2
    gamma_2 = 1.0 / math.sqrt(1.0 - norm_v2_sq / (c**2))

    # beta vector v2
    beta_x = (1.0/c)*(v1[0] - v2[0] + (1.0/gamma_2)*v2_perp_x) / (1.0 - (v2_dot_v1)/(c**2))
    beta_y = (1.0/c)*(v1[1] - v2[1] + (1.0/gamma_2)*v2_perp_y) / (1.0 - (v2_dot_v1)/(c**2))
    beta_norm_sq = beta_x*beta_x + beta_y*beta_y #without dot_product because not array
 
    # np.dot(r2-r1)/(norm(r2-r1)**2)
    dr_x = r2[0] - r1[0]
    dr_y = r2[1] - r1[1]
    norm_dr = math.sqrt(dr_x*dr_x + dr_y*dr_y) #without dot_product because not array
    r_vers_x = dr_x / norm_dr
    r_vers_y = dr_y / norm_dr

    #Lorentz factor v1-v2
    gamma = math.sqrt(1.0 / (1.0 - beta_norm_sq))

    # Beta dot r_vers
    beta_dot_r_vers = beta_x*r_vers_x + beta_y*r_vers_y

    # Doppler factor
    D = 1.0 / (gamma * (1.0 - beta_dot_r_vers))

    return D

@cuda.jit(device=True)
def dot_product(a, b):
    result = 0.0
    for i in range(a.size):
        result += a[i] * b[i]
    return result

@cuda.jit(device=True)
def radiative_efficiency(chi):
    Z1 = 1. + (1. - chi**2)**(1/3) * ((1. + chi)**(1/3) + (1. - chi)**(1/3))
    Z2 = math.sqrt(3. * chi**2 + Z1**2)
    Eisco = (4. - chi * Z1 - math.sqrt(3 * Z2 - 2. * Z1)) / (3. * math.sqrt(3))
    eta = 1.- Eisco
    return eta

@cuda.jit(device=True)
def L_edd(M):
    return 4.*math.pi*G*M*c/0.1
    
@cuda.jit(device=True)
def temperature(pos, M, f_edd, chi, R_in):
    R = math.sqrt(pos[0]**2+pos[1]**2)
    eta = radiative_efficiency(chi)
    M_dot = f_edd*L_edd(M)/(eta*c**2)
    return ((3.*G*M*M_dot)/(8.*math.pi*sigma_B*R**3))**(1/4)*(1.-math.sqrt(R_in/R))**(1/4)  

In [4]:
@cuda.jit(device=True)
def compute_planck_spectrum(nu, T):
    num = 2.*h*nu**3/c**2
    den = math.exp(h*nu/(k*T)) - 1.0
    return num /den

@cuda.jit(device=True)
def compute_ionizing_flux(T, D, z, pos, dr, dtheta):
    nu_min = 13.6*1.60218e-19/h
    nu_max = 20.*13.6*1.60218e-19/h
    N = int(1e5)  # Number of integration steps
    dnu = (nu_max-nu_min)/N
    flux = 0.0
    
    T_eff = D*T/(1. + z)

    for i in range(N):
        nu = nu_min + i * dnu
        flux += compute_planck_spectrum(nu, T_eff)*dnu

    return flux*math.sqrt(pos[0]**2+pos[1]**2)*dr*dtheta

In [5]:
@cuda.jit
def sum_kernel(d_r, d_angle, d_time, d_flux, d_flux_def):
  idx_t = cuda.grid(1)
  if idx_t < d_time.size:
    for i_x in range(d_r.size):
          for i_fi in range(d_angle.size):
            d_flux_def[idx_t] = d_flux_def[idx_t] + d_flux[idx_t][i_fi][i_x]


In [6]:
@cuda.jit(device=True)
def find_temperature(gp_times, starting_light_curve, t):#, idx_angle, idx_R):
    lower_diff = 1e10
    upper_diff = 1e10
    lower_idx = 0  # initialize to valid index 0
    upper_idx = gp_times.size - 1  # initialize to last valid index
    y_predicted = 0.

    for i in range(gp_times.size):
        val = gp_times[i]
        diff = abs(val - t)

        if val <= t and diff < lower_diff:
            lower_diff = diff
            lower_idx = i
        if val >= t and diff < upper_diff:
            upper_diff = diff
            upper_idx = i
            
        y_predicted = (starting_light_curve[lower_idx] + starting_light_curve[upper_idx]) / 2.

    return y_predicted

In [7]:
def radiative_efficiency_cpu(chi):
    Z1 = 1 + (1 - chi**2)**(1/3) * ((1 + chi)**(1/3) + (1 - chi)**(1/3))
    Z2 = math.sqrt(3 * chi**2 + Z1**2)
    Eisco = (4 - chi * Z1 - math.sqrt(3 * Z2 - 2 * Z1)) / (3 * math.sqrt(3))
    eta = 1 - Eisco
    return eta

def L_edd_cpu(M):
    return 4*math.pi*G*M*c/0.1
    
def Temperature(R, M, f_edd, chi, R_in):
    #R = math.sqrt(pos[0]**2+pos[1]**2)
    eta = radiative_efficiency_cpu(chi)
    M_dot = f_edd*L_edd_cpu(M)/(eta*c**2)
    return ((3*G*M*M_dot)/(8*math.pi*sigma_B*R**3))**(1/4)*(1-math.sqrt(R_in/R))**(1/4)  

# Function to generate a set of temperatures using Gaussian processes
def generate_set_of_temperatures(times, Rs, thetas, M, f_edd, chi, R_in, tau, sigma):
    """
    Generate a set of Gaussian Process (GP) objects and initial temperature samples 
    for a grid of radii (Rs) and angles (thetas) over specified time points.

    Args:
        times (array-like): Array of time points at which the GP is computed.
        Rs (array-like): Array of radii values representing the grid's columns.
        thetas (array-like): Array of angular values (in radians) representing the grid's rows.
        M (float): Mass parameter used in the temperature calculation.
        f_edd (float): Eddington factor used in the temperature calculation.
        chi (float): Opacity parameter used in the temperature calculation.
        R_in (float): Inner radius parameter used in the temperature calculation.
        tau (float): Timescale parameter for the GP kernel.
        sigma (float): Amplitude parameter for the GP kernel.

    Returns:
        tuple: A tuple containing:
            - gps (list of lists): A 2D list of GP objects, where rows correspond to 
              angular values (thetas) and columns correspond to radii (Rs).
            - starting_ys (list of lists): A 2D list of initial temperature samples 
              generated from the GPs, with the same structure as `gps`.

    Notes:
        - The grid is structured such that the outer list corresponds to rows (thetas),
          and the inner list corresponds to columns (Rs).
        - Each GP object is initialized with a kernel defined by the given `tau` and `sigma`
          parameters and a mean temperature based on the radius `Rs[i]`.
    """
    # Initialize Gaussian Process (GP) objects and starting temperature samples
    gps = [[[] for _ in range(len(Rs))] for _ in range(len(thetas))]
    starting_ys = [[[] for _ in range(len(Rs))] for _ in range(len(thetas))]
    for i in range(len(thetas)):
        for j in range(len(Rs)):
            # Define a GP kernel with specified parameters
            kernel_ = terms.RealTerm(log_a=np.log(sigma), log_c=-np.log(tau))
            # Create a GP object with the kernel and mean temperature at the given radius
            # Here the radius is the mean radius of the element
            gp = celerite.GP(kernel_, mean=np.log10(Temperature(Rs[j]+(Rs[1]-Rs[0])/2, M, f_edd, chi, R_in)))
            gp.compute(times)  # Precompute the GP for the given time points
            starting_ys[i][j] = gp.sample()  # Sample initial temperatures
            gps[i][j] = gp  # Store the GP object
    return np.transpose(10.0**np.array(starting_ys), (2, 0, 1)) #to have [idx_t, idx_fi, idx_r] #,gps

def approximate_temperature(t, starting_light_curve, times):
    """
    Approximate the temperature at a given time `t` using a simple linear interpolation
    between the nearest time points in the `times` array.

    Args:
        t (float): The time at which to approximate the temperature.
        starting_light_curve (array-like): The light curve values corresponding to `times` for a given accretion disc element.
        times (array-like): The array of time points used to create the initial lightcurves.

    Returns:
        float: The approximated temperature at time `t`.
    """
    # Find the indices of the time points just before and after the given time `t`
    index1 = np.where(times < t)[-1]  # Indices where times are less than `t`
    index2 = np.where(times > t)[0]  # Indices where times are greater than `t`

    # Perform a simple average of the light curve values at the two nearest time points
    y_predicted = (starting_light_curve[index1[-1]] + starting_light_curve[index2[0]]) / 2

    return y_predicted

In [8]:
# Function to calculate the Planck spectrum for a given temperature
def planck_spectrum(nu, T):
    return (2 * h * nu**3) / (c**2) * (1 / (np.exp(h * nu / (k * T)) - 1))

# Function to calculate the ionizing flux for a given temperature
# The temperature here should be D*T/(1+z) where D is the Doppler factor, 1+z is the gravitational redshift and T is the rest frame wavelength 
def ionizing_flux_element(T,D,R,dr,dtheta):
    def integrand(nu):
        return planck_spectrum(nu, T)
    flux, _ = quad(integrand, 13.6 / h * 1.60218e-19, 20 * 13.6 / h * 1.60218e-19)
    return flux*R*dr*dtheta

In [9]:
@cuda.jit()
def accretion_disk_kernel(angle, R, M, times, gp_times, d_temp_curves, initial_phase, v_obs, r_obs, d_debug):
    idx_times, idx_angle, idx_R = cuda.grid(3)
    
    #starting_light_curve = cuda.shared.array(d_temp_curves, dtype=float64)
    if idx_times < times.size and idx_angle < angle.size and idx_R < R.size:
        j = idx_angle
        k = idx_R
        r = R[idx_R]
        theta = angle[idx_angle]
        gp_temp = d_temp_curves[:, idx_angle, idx_R]
        t = times[idx_times]

        theta_prime = de_evolve(float64(r), float64(theta), float64(t), float64(initial_phase), float64(M))

        positions = position_components(theta_prime, r) #returns (x,y), if check you need d_debug = positions[i]
        velocities = velocity_components(theta_prime, r, M, 0.)
        doppler = doppler_factor(v_obs, velocities, r_obs, positions)
        T = find_temperature(gp_times, gp_temp, t)
        T_eff = T*doppler/(1+z)
        if T_eff < 1e8:
            flux = math.exp(-1.06581189e+02 +5.39800294e+01*math.log10(T_eff) -8.69566619e+00*math.log10(T_eff)**2+ 6.02830718e-01*math.log10(T_eff)**3-1.43840088e-02*math.log10(T_eff)**4)
        #T = temperature(positions, M, f_edd, chi, R_in)
        else:
            flux = math.exp(8.49396521e+00+1.76942309e+00*math.log10(T_eff)-1.12695341e-01*math.log10(T_eff)+7.33461259e-03*math.log10(T_eff)**3-1.78956502e-04*math.log10(T_eff)**4)
        d_debug[idx_times, idx_angle, idx_R] = flux*r*dr*dtheta#compute_ionizing_flux(T,doppler,z, positions,dr,dtheta)

In [10]:
# Defining constants
h = 6.62607015e-34  # Planck's constant (Joule-seconds)
c = 3e8  # Speed of light (m/s)
k = 1.380649e-23  # Boltzmann constant (Joule/Kelvin)
G = 6.67e-11  # Gravitational constant (m^3/kg/s^2)
sigma_B = 1.28e-23  # Stefan-Boltzmann constant (W/m^2/K^4)
M_sun = np.float64(2e30)  # Solar mass (kg)

M = np.float64(1e8*M_sun)
d_M = cuda.to_device(M)
inc = np.radians(30.)
chi = 0.8
R_in = np.float64(1.5*R_ISCO(M, chi))
f_edd = 0.5
z = 0.1
tau = 100.
sigma = 0.5

r = np.linspace(1.1*R_in,3.0*R_in, 20).astype(np.float64) #if r_1 = R_in Nan for high chi, check
#r = np.linspace(100,200).astype(np.float64)
angle = np.linspace(0., 2.*np.pi, 20).astype(np.float64)
times = np.linspace(20., 40., 20).astype(np.float64)
gp_times = np.linspace(0., 50., 1000).astype(np.float64)

dr = r[1]-r[0]
dtheta = angle[1]-angle[0]
d_r = cuda.to_device(r)
d_angle = cuda.to_device(angle)
d_times = cuda.to_device(times)
d_gp_times = cuda.to_device(gp_times)
r_obs = cuda.to_device([5.0*R_in,5.0*R_in])
#r_obs = cuda.to_device([100,100])
v_obs = cuda.to_device([c/2,c/2])

deb = np.zeros([times.size, angle.size, r.size])
d_deb = cuda.to_device(deb)
deb_def = np.zeros([times.size])
d_deb_def = cuda.to_device(deb_def)

In [11]:
start_time = tempo.time()
temp_curves = generate_set_of_temperatures(gp_times, r, angle, M, f_edd, chi, R_in, tau, sigma)
temp_curves = np.ascontiguousarray(temp_curves)
d_temp_curves = cuda.to_device(np.ascontiguousarray(temp_curves))
end_time = tempo.time()
print(f'Expected shape: ({len(gp_times):.0f}, {len(angle):.0f}, {len(r):.0f})')
print('Resulting shape', d_temp_curves.shape)
print('Tempo per generare curve di temperatura:', end_time-start_time)

Expected shape: (1000, 20, 20)
Resulting shape (1000, 20, 20)
Tempo per generare curve di temperatura: 0.08019566535949707


In [12]:
threads_per_block =  (4)
blocks_per_grid = (
      (d_times.size + (threads_per_block - 1)) // threads_per_block,
  )

In [13]:
threads_per_block_1 =  (4,4,4)
blocks_per_grid_1 = (
      (d_times.size + (threads_per_block_1[0] - 1)) // threads_per_block_1[0],
      (d_angle.size + (threads_per_block_1[1] - 1)) // threads_per_block_1[1],
      (d_r.size + (threads_per_block_1[2] - 1)) // threads_per_block_1[2]
  )

In [14]:
start_time_gpu = tempo.time()
accretion_disk_kernel[blocks_per_grid_1, threads_per_block_1](d_angle, d_r, M, d_times, d_gp_times, d_temp_curves,  0., v_obs, r_obs, d_deb)
cuda.synchronize()
sum_kernel[blocks_per_grid, threads_per_block](d_r, d_angle, d_times, d_deb, d_deb_def)
cuda.synchronize()
end_time_gpu_without_copying = tempo.time()

/leonardo_scratch/large/userexternal/esottoco/box/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 125 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


/leonardo_scratch/large/userexternal/esottoco/box/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [15]:
deb = d_deb.copy_to_host()
print(f'Expected shape: ({len(times):.0f}, {len(angle):.0f}, {len(r):.0f})')
print('Resulting shape', deb.shape)
end_time_gpu = tempo.time()

Expected shape: (20, 20, 20)
Resulting shape (20, 20, 20)


In [16]:
deb_def = d_deb_def.copy_to_host()
print(f'Expected shape: ({len(times):.0f})')
print('Resulting shape', deb_def.shape)

Expected shape: (20)
Resulting shape (20,)


In [17]:
D = []
angl = []
vx = []
vy = []
x = []
y = []
T = []
flux = []
r1= r_obs.copy_to_host()#np.array([100., 100.])
v1 = v_obs.copy_to_host()#np.array([c/2, c/2])

start_time_cpu = tempo.time()
for t in times:
    for idx_ang, ang in enumerate(angle):
        for idx_r, erre in enumerate(r):
            td = erre*(1+math.sin(inc)*math.cos(ang))
            kf = math.sqrt(G * M / erre**3)
            ##phi - 2*math.pi * math.floor(phi / (2*math.pi))
            #(theta - (time -td)*kf + initial_phase) - (2.*math.pi)*math.floor((theta - (time -td)*kf + initial_phase)/(2.*math.pi))

            ang_pre = (ang - (t -td)*kf + 0.) - (2*math.pi) * math.floor((ang - (t -td)*kf + 0.)/(2*math.pi)) 
            angl.append(ang_pre)
            v_kep = math.sqrt(G * M / erre)  # Keplerian velocity
            vx.append(v_kep*np.cos(ang_pre))
            vy.append(-v_kep*np.sin(ang_pre))
            x.append(erre * math.sin(ang_pre))
            y.append(erre*math.cos(ang_pre))

            r2 = np.array([erre * math.sin(ang_pre), erre * math.cos(ang_pre)]) 
            v2 = np.array([v_kep*np.cos(ang_pre), -v_kep*np.sin(ang_pre)])
            
            v2_para = np.dot(v2, v1) * v1 / np.linalg.norm(v1)**2  # Parallel component of v2 
            v2_perp = v2 - v2_para  # Perpendicular component of v2
            gamma_2 = 1 / np.sqrt(1 - np.linalg.norm(v2)**2 / c**2)  # Lorentz factor for v2
            beta = 1 / c * (v1 - v2 + 1 / gamma_2 * v2_perp) / (1 - np.dot(v1, v2) / c**2)
            r_vers = (r2 - r1) / np.linalg.norm(r2 - r1)  # Unit vector in the direction of r2 - r1
            gamma = np.sqrt(1 / (1 - np.linalg.norm(beta)**2))
            dop = 1 / (gamma * (1 - np.dot(beta, r_vers)))
            D.append(dop)  # Doppler factor

            #Z1 = 1 + (1 - chi**2)**(1/3) * ((1 + chi)**(1/3) + (1 - chi)**(1/3))
            #Z2 = np.sqrt(3 * chi**2 + Z1**2)
            #Eisco = (4 - chi * Z1 - np.sqrt(3 * Z2 - 2 * Z1)) / (3 * np.sqrt(3))
            #eta = 1 - Eisco
            #M_dot = f_edd * 4*np.pi*G*M * c / 0.1 / (eta * c**2)
            #temp = ((3 * G *M*M_dot) / (8 * np.pi * sigma_B * erre**3))**(1/4) * (1 - np.sqrt(R_in / erre))**(1/4)
            temp = approximate_temperature(t, temp_curves[:, idx_ang, idx_r], gp_times)
            T.append(temp)
            f = ionizing_flux_element(dop*temp/(1+z), dop, erre,dr,dtheta)
            flux.append(f)

end_time_cpu = tempo.time()

angl = np.array(angl)
angl = angl.reshape(len(times), len(angle), len(r))
y = np.array(y)
x = np.array(x)
x = x.reshape(len(times), len(angle), len(r))
y = np.array(y)
y = y.reshape(len(times), len(angle), len(r))
vx = np.array(vx)
vx = vx.reshape(len(times), len(angle), len(r))
vy = np.array(vy)
vy = vy.reshape(len(times), len(angle), len(r))
D = np.array(D)
D = D.reshape(len(times), len(angle), len(r))
T = np.array(T)
T = T.reshape(len(times), len(angle), len(r))
flux = np.array(flux)
flux = flux.reshape(len(times), len(angle), len(r))
flux_def = np.array(np.sum(np.sum(flux, axis = 2), axis=1))
print("Total elements in f_def:", flux_def.shape)
print("Expected elements:", len(times))

Total elements in f_def: (20,)
Expected elements: 20


In [18]:
check = 'f' #choices = ['ang','x', 'y', 'vx', 'vy', 'D', 'T', 'DT', 'f']
choice_map = {
    'angl': angl,
    'x': x,
    'y': y,
    'vx': vx,
    'vy': vy,
    'D': D,
    'T': T,
    'DT': D * T,
    'f': flux
}

deb_cpu = choice_map.get(check)

In [19]:
i = np.random.randint(deb.shape[0])
j = np.random.randint(deb.shape[1])
k = np.random.randint(deb.shape[2])
print('----------------------------- Check '+check+' --------------------------------------')
print('GPU: ', deb[i, j, k])
print('CPU: ', deb_cpu[i, j, k])

----------------------------- Check f --------------------------------------
GPU:  6.175797255528916e+33
CPU:  2.243637026787437e+41


In [20]:
check_def = True
if check_def:
    i = np.random.randint(deb_def.shape[0])
    print('----------------------------- Final Check --------------------------------------')
    print('GPU: ', deb_def[i])
    print('CPU: ', flux_def[i])

----------------------------- Final Check --------------------------------------
GPU:  2.20823898434277e+37
CPU:  3.054936113652794e+44


In [21]:
print("Any infs?", np.isinf(flux_def).any())
print("Positive infs:", np.where(flux_def == np.inf)[0].shape)
print("Negative infs:", np.where(flux_def == -np.inf)[0].shape)

Any infs? False
Positive infs: (0,)
Negative infs: (0,)


In [22]:
print("Any infs?", np.isinf(deb_def).any())
print("Positive infs:", np.where(deb_def == np.inf)[0].shape)
print("Negative infs:", np.where(deb_def == -np.inf)[0].shape)

Any infs? False
Positive infs: (0,)
Negative infs: (0,)


In [23]:
print('---------------------- Check Tempistiche -----------------------------')
print('Tempo impiegato con GPU:', end_time_gpu_without_copying-start_time_gpu)
print('Tempo impiegato con GPU+copia su CPU:', end_time_gpu-start_time_gpu)
print('Tempo impiegato con CPU:', end_time_cpu-start_time_cpu)

---------------------- Check Tempistiche -----------------------------
Tempo impiegato con GPU: 0.8327937126159668
Tempo impiegato con GPU+copia su CPU: 0.8375489711761475
Tempo impiegato con CPU: 0.5656228065490723


In [24]:
with open('error_time_evaluation/output_N_1e5.txt', 'w') as f:
    f.write(f'GPU\t{end_time_gpu-start_time_gpu:.2f}\n')
    f.write(f'CPU\t{end_time_cpu-start_time_cpu:.2f}\n')
    for a, b in zip(deb_def, flux_def):
        f.write(f"{a}\t{b}\n") 